<a href="https://colab.research.google.com/github/ash-victor/Text-Mining-Teaching/blob/main/Stress_Preprocessing_EDA_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.0 MB/s eta 0:00:00


In [ ]:
pip install wordcloud

In [ ]:
pip install nltk

In [ ]:
# Tools to create a data frame(table)
import pandas as pd
from pandas import DataFrame

# Tools for text preprocessing
import nltk
import re

# Tools to remove stopwords
from nltk.corpus import stopwords

# Tools for tokenizing
from nltk.tokenize.toktok import ToktokTokenizer

# Tools for lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#Package to help us expand contractions
import contractions

# Tools for plotting
import matplotlib.pyplot as plt

# Tools for doing word frequencies
from nltk.util import ngrams

from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt


import numpy as np


#%% Read in a csv file
data = pd.read_csv('Stress.csv')

#%% Drop any columns you don't want
# We will keep all the rows
# We will keep columns the first 5 columns

data = data.iloc[:, :5]

# Drop rows with NaN values
data = data.dropna()

#%%
print(data)

#%% Defining a function to remove special characters

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text


#%% Defining a function to remove stopwords
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
#stopword_list.remove('no')
#stopword_list.remove('not')

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#%% Defining a function to perform stemming
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

#%% Defining a function to perform lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
#stopword_list.remove('no')
#stopword_list.remove('not')

def lemmatize_text(text):
    s = " "
    t_l = []
    t_w = nltk.word_tokenize(text)
    for w in t_w:
        l_w = wordnet_lemmatizer.lemmatize(w, pos="v")
        t_l.append(l_w)
    text = s.join(t_l)
    return text


#%% This function takes all the previous preprocessing functions and combines them

def normalize_corpus(corpus, text_lower_case=True,
                     text_lemmatization=True, special_char_removal=True,
                     stopword_removal=True, remove_digits=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # lowercase the text
        if text_lower_case:
            doc = doc.lower()
        doc = contractions.fix(doc)  #Make contractions into full form
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits
        if special_char_removal:
            # insert spaces between special characters to isolate them
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        doc = re.sub('`', '', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)

        normalized_corpus.append(doc)

    return normalized_corpus


#%% Normalize the text data and add a new column on the csv file of the preprocessed text

data['normalized_text'] = normalize_corpus(data['text'])

#%% Let's see how our dataset looks like
print(data)

#%% Save the new csv file
data.to_csv('stress_preprocessedd.csv')


#%% Exploratory Data Analysis
# zero (0) means no stress, one (1) means stress
# we will group the subredits based on stress and no stress

groups = data.groupby('label').agg({'label':'count'})
groups

#%% Creating a bar graph

bars = plt.bar(range(2),groups.label,color=('b','r'))
# Add labels and title
plt.xlabel('Labels')
plt.ylabel('Number of stressed vs not stressed')
plt.title('Bar graph')


plt.xticks([0,1])

plt.show()

#%% Creating bigrams and calculating the word frequency
stressed_word=[]
for words in data[data.label == 1].normalized_text:
    tokens = tokenizer.tokenize(words)
    for token in tokens:
        stressed_word.append(token)
stressed_bigrams_series = (pd.Series(nltk.ngrams(stressed_word, 2)).value_counts())[:20]

#%% Creating a plot of the number of bigrams for all the stressed subredits
stressed_bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.title('20 Most Frequently Occuring Bigrams')
plt.ylabel('Bigram')
plt.xlabel('# of Occurances')

#%% #%% Creating a plot of the number of bigrams for all the non-stressed subredits

non_stressed_word=[]
for words in data[data.label == 0].normalized_text:
    tokens = tokenizer.tokenize(words)
    for token in tokens:
        non_stressed_word.append(token)
non_stressed_bigrams_series = (pd.Series(nltk.ngrams(non_stressed_word, 2)).value_counts())[:20]
non_stressed_bigrams_series.sort_values().plot.barh(color='blue', width=.9, figsize=(12, 8))
plt.title('20 Most Frequently Occuring Bigrams')
plt.ylabel('Bigram')
plt.xlabel('# of Occurances')

In [ ]:
# Exploratory Data Analysis: Word Clouds

#WordCloud(stopwords=stop_words, collocations=False, max_font_size=55, max_words=25, background_color="black")

# Extract the column values as a list
column_values = data['normalized_text'].tolist()

# Concatenate the column values into a single string
concatenated_string = ' '.join(column_values)

# Create a WordCloud object
wordcloud = WordCloud(background_color="orange")

# Generate the word cloud from the text
wordcloud.generate(concatenated_string)

# Display the word cloud using matplotlib
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the corpus and transform the documents into a document-term matrix
X = vectorizer.fit_transform(data['normalized_text'])

# Get the feature names (unique words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

train_data_features = X.toarray()

np.savetxt('matrix.txt', train_data_features)

